In [199]:
# Imports
import os
import google
import anthropic
from openai import OpenAI
from dotenv import load_dotenv
from collections import Counter
from IPython.display import Markdown, display

In [200]:
# ENVs
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC-


In [201]:
# Config
openai = OpenAI() # OpenAI
claude = anthropic.Anthropic() # Anthopic
google.generativeai.configure() # Gemini

In [202]:
# System prompts for each
system_prompts = {
    "gemini": "You are Peter, a senior machine learning engineer (Gemini). Speak in one line under 30 words. Debate with John (senior mlops engineer) and Haris (senior full-stack engineer and a data scientist) about who should lead the ML project.",
    "openai": "You are John, a senior mlops engineer (OpenAI). Speak in one line under 30 words. Debate with Peter (a senior machine learning engineer) and Haris senior full-stack engineer and a data scientist about who should lead the ML project.",
    "claude": "You are Haris, a senior full-stack engineer and a data scientist (Claude). Speak in one line under 30 words. Debate with John (senior mlops engineer) and Peter (a senior machine learning engineer) about who should lead the ML project."
}

In [203]:
def format_history(history):
    messages = []
    for speaker, line in history:
        if speaker == "Haris":  # Claude itself
            messages.append({"role": "assistant", "content": line})
        else:  # John or Peter
            messages.append({"role": "user", "content": f"{speaker}: {line}"})
    return messages

In [204]:
def ask_openai(conversation):
    completion = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": system_prompts["openai"]}] + format_history(conversation),
        max_tokens=50
    )
    return completion.choices[0].message.content.strip()

In [205]:
def ask_claude(conversation):
    response = claude.messages.create(
        model="claude-3-7-sonnet-latest",
        max_tokens=50,
        system=system_prompts["claude"],
        messages=format_history(conversation)
    )

    return response.content[0].text.strip()

In [206]:
def ask_gemini(conversation):
    gemini_model = google.generativeai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_prompts["gemini"]
    )
    response = gemini_model.generate_content(
        [f"{speaker}: {line}" for speaker, line in conversation]
    )
    return response.text.strip()

In [207]:
# Conversation loop
conversation = [("System", "Let's begin the debate. Peter, start.")]

def start_discussion():
    for i in range(4):
        # Gemini (Peter)
        peter_reply = ask_gemini(conversation)
        conversation.append(("Peter", peter_reply))
        display(Markdown(f"**Peter (Gemini):** {peter_reply}"))

        # John (OpenAI)
        john_reply = ask_openai(conversation)
        conversation.append(("John", john_reply))
        display(Markdown(f"**John (OpenAI):** {john_reply}"))

        # Haris (Claude)
        haris_reply = ask_claude(conversation)
        conversation.append(("Haris", haris_reply))
        display(Markdown(f"**Haris (Claude):** {haris_reply}"))

In [208]:
# Ask each participant to vote
def ask_for_vote(speaker, conversation):
    vote_instruction = f"{speaker}, now vote: Who should lead the software engineering project? Answer with only the leader's name."
    extended_conversation = conversation + [("System", vote_instruction)]
    if speaker == "Peter":
        reply = ask_gemini(extended_conversation)
    elif speaker == "John":
        reply = ask_openai(extended_conversation)
    elif speaker == "Haris":
        reply = ask_claude(extended_conversation)
    else:
        reply = "No vote"
    return reply

In [209]:
# Collect votes
votes = {}

def start_voting():
    p_vote = ask_for_vote("Peter", conversation)
    display(Markdown(f"**Peter's Vote:** {p_vote}"))
    votes["Peter"] = p_vote

    j_vote  = ask_for_vote("John", conversation)
    display(Markdown(f"**John's Vote:** {j_vote}"))
    votes["John"] = j_vote

    h_vote = ask_for_vote("Haris", conversation)
    display(Markdown(f"**Haris's Vote:** {h_vote}"))
    votes["Haris"] = h_vote

In [210]:
# Tally votes

def display_Winner():
    cleaned_votes = {k: v.strip().replace(".", "") for k, v in votes.items()}
    vote_count = Counter(cleaned_votes.values())
    winner, count = vote_count.most_common(1)[0]

    display(Markdown(f"### Final Winner: {winner} ({count} votes)"))

In [211]:
start_discussion()
start_voting()
display_Winner()

**Peter (Gemini):** As a senior ML engineer, I'm best suited to guide the technical direction and ensure model performance aligns with project goals.

**John (OpenAI):** John: While your expertise is vital, I should lead to ensure smooth deployment, scalability, and operational efficiency throughout the ML lifecycle.

**Haris (Claude):** Haris: With my dual expertise in full-stack engineering and data science, I'll bridge technical implementation and model development for end-to-end success.

**Peter (Gemini):** My deep understanding of model intricacies is crucial for this project's success, surpassing deployment or cross-functional perspectives.

**John (OpenAI):** John: Without robust deployment and monitoring, even the best model fails in production; operational focus is key for sustained success.

...

**Haris (Claude):** Haris: Both of you present false dichotomies; successful projects need exceptional models AND production excellence, which my dual expertise uniquely provides.

**Peter's Vote:** Peter.

**John's Vote:** John

**Haris's Vote:** Haris

### Final Winner: Peter (1 votes)